In [1]:
import autograd.numpy as np
import autograd.numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
num_sequences = 50
time_length = 100
image_length = 20
generate_dataset_filename = './bouncingball'
dataset_filename = './bouncingball.npy'

## Generate Dataset

In [3]:
def generate_video(image_length, time_length):
    video = np.empty([image_length, time_length])
    rand_pt = npr.randint(image_length)
    rand_dir = npr.randint(2) ## 0 for downwards, 1 for upwards
    for i in range(time_length):
        one_hot = np.zeros(image_length)
        one_hot[rand_pt] = 1.0
        video[:, i] = one_hot
        if rand_pt == 0:
            rand_dir = 0
        if rand_pt == 19:
            rand_dir = 1

        if rand_dir == 0:        
            rand_pt += 1
        else:
            rand_pt -= 1
    return video

def generate_dataset(generate_dataset_filename, num_sequences, image_length, time_length):
    video_lists = []
    for i in range(num_sequences):
        video = generate_video(image_length, time_length)
#         plt.imshow(video, cmap='gray', vmin=0, vmax=1)
#         plt.colorbar()
        video_lists.append(video.reshape(1, image_length * time_length))
    video_arrays = np.concatenate(video_lists, 0)
    np.save(generate_dataset_filename, video_arrays)
    return video_arrays

video_arrays = generate_dataset(generate_dataset_filename, num_sequences, image_length, time_length)



## Load Dataset

In [17]:
def load_data(dataset_filename, num_sequences, image_length, time_length):
    train_dataset = np.load(dataset_filename).reshape(num_sequences, image_length, time_length)
#     for i in range(num_sequences):
#         plt.imshow(train_dataset[i, :, :], cmap='gray', vmin=0, vmax=1)
#         plt.show()
    x, y, z = train_dataset.shape
    print('dataset size : %d, image length : %d, sequence length : %d' % (x, y, z))
    return train_dataset

train_dataset = load_data(dataset_filename, num_sequences, image_length, time_length)

dataset size : 50, image length : 20, sequence length : 100
